# Loading and Wrangling Data

.NET Interactive offers far more than just interactive notebooks for executing trivial code. It exposes a rich API for creating formatters, magic commands and additional sub-language-kernels. The `Microsoft.DotNet.Interactive.ExtensionLab` package showcases all of those features.

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab, *-*"

Run this cell if you don't already have the SQLite data base containing the energy data set. It contains energy load data for a lot of European national grids. The European market is pretty interconnected and because of the energy market liberation they have to rather open with certain subsets of their data. Not all of the countries are totally compliant but the Austrian subset is rather tidy.

In [ ]:
open System
open System.Net.Http
open System.IO

let client = new HttpClient()

let dataPackage =
    client.GetAsync("https://data.open-power-system-data.org/time_series/2020-10-06/time_series.sqlite")
    |> Async.AwaitTask
    |> Async.RunSynchronously

let copyToTarget (targetPath: string) (msg: HttpResponseMessage) =
    use fileStream = new FileStream(targetPath, FileMode.CreateNew)
    use contentStream = msg.Content.ReadAsStream()
    contentStream.CopyTo(fileStream)

let sqliteDbPath = "../data/time_series.sqlite"

copyToTarget sqliteDbPath dataPackage

The Extension Lab contains a SQLite extension which allows us to use the interactive notebook as an SQL interface. If our data would reside in SQL Server we could use this "natively" in .NET Interactive without loading an extension. In this case SQLite is fine for us. Every database connection we want to use can be named and referenced in the following cells.

In [ ]:
#!connect sqlite --kernel-name energytimeseries "Data Source=../data/time_series.sqlite"

And as you can see all the SQLite querying works as expected.

In [ ]:
#!sql-energytimeseries
select name from sqlite_master
where type in ('table', 'view')
and name not like 'sqlite_%'
order by 1;

It makes no difference if we write queries, mutate the schema or call internal stored procedure. Everything pretty much behaves as it would in a normal SQLite console. With one difference: we can use custom formatters (in this case the Nteract data viewer) to make our experience more joyful than staring at ASCII tables.

In [ ]:
#!sql-energytimeseries
select name from PRAGMA_TABLE_INFO("time_series_60min_singleindex");

In [ ]:
#!sql-energytimeseries
select utc_timestamp, AT_load_actual_entsoe_transparency
from time_series_60min_singleindex
where AT_load_actual_entsoe_transparency is not null
limit 1000;

In [ ]:
#!sql-energytimeseries
select utc_timestamp, AT_load_actual_entsoe_transparency
from time_series_15min_singleindex
where AT_load_actual_entsoe_transparency is not null
limit 4000;

In [ ]:
#!sql-energytimeseries
select count(1)
from time_series_60min_singleindex
where AT_load_actual_entsoe_transparency is not null;

In [ ]:
#!sql-energytimeseries
select utc_timestamp, cast(strftime('%Y', utc_timestamp) as int) as yr, AT_load_actual_entsoe_transparency
from time_series_60min_singleindex
where AT_load_actual_entsoe_transparency is not null
and yr < 2020
and yr > 2017;

Because we're still in the .NET Interactive Kernel we can switch to another language (like F# in this case) and keep on working in the same notebook. This is nice because I like to seperate my notebooks by topic and not by language.

In [ ]:
#r "nuget: Microsoft.Data.Sqlite, 5.0.5"
#r "nuget: Ply, 0.3.1"

We could be using Dapper, Entity Framework or any other .NET library which speaks SQL. As our use case is extremely simple I opt for the good old ADO.NET.

In [ ]:
open Microsoft.Data.Sqlite
open FSharp.Control.Tasks.NonAffine

let connectionString = "Data Source=../data/time_series.sqlite"

type RowRecord =
    { TimeStamp: DateTime
      Value: int }

let getSeries () = task {
    use connection = new SqliteConnection(connectionString)
    do! connection.OpenAsync()

    let command = connection.CreateCommand()
    command.CommandText <-
        """
        select utc_timestamp, cast(strftime('%Y', utc_timestamp) as int) as yr, AT_load_actual_entsoe_transparency
        from time_series_60min_singleindex
        where AT_load_actual_entsoe_transparency is not null
        """

    use! reader = command.ExecuteReaderAsync()
    let rows = ResizeArray<RowRecord>()
    while reader.Read() do
        let newRecord = { TimeStamp = reader.GetDateTime(0); Value = reader.GetInt32(2) }
        rows.Add(newRecord)

    return (Seq.toList rows)
}

let austriaLoad =
    getSeries()
    |> Async.AwaitTask
    |> Async.RunSynchronously

display (List.length austriaLoad)

austriaLoad
|> List.take 10

If we compare the output of the last cell and the length of the list we see, that we get the same results. Exactly as it should be.

In [ ]:
austriaLoad
|> List.distinct
|> List.length

Looking at the amount of distinct timestamps we see a little discrepancy. We could stay in the world of lists and sequences to find it out but as I'd like to wrangle a little bit more anyway this would be a great opportunity to introduce `Deedle`.

In [ ]:
let austriaLoadUnique =
    austriaLoad
    |> List.distinctBy (fun row -> row.TimeStamp)

austriaLoadUnique
|> List.length

In [ ]:
#r "nuget: Deedle, 2.3.0"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha6"

`Deedle` is F#'s implementation of a Data Frame. You might have used something like that with pandas or with R's native `data.frame`. `Deedle` is just that, a library that lets you work with "rectangular" data, combinsing tables out of different series.

In [ ]:
open Deedle

let data =
    austriaLoadUnique
    |> Frame.ofRecords
    |> Frame.indexRowsDate "TimeStamp"

data

`Deedle` is pretty good for tasks related to time series. In this case I want to see if Ihave gaps in my time series. I construct a complete index of timestamps from the first timestamp to the last one, add a running counter to model a simple linear trend and join the frames. The exact lookup will create `NA` values wherever there is a gap.

In [ ]:
let minDate =
    data.RowKeys
    |> Seq.min

let maxDate =
    data.RowKeys
    |> Seq.max

let hoursBetween = (maxDate - minDate).TotalHours
let numOfValues = hoursBetween + 1.

let timeSeriesIndex = [ for x in 0. .. hoursBetween -> minDate.AddHours(x)]
let tsTicks = [ 1. .. numOfValues ]

let frameSeries =
    Seq.zip timeSeriesIndex tsTicks
    |> Series.ofObservations

let baseFrame = Frame.ofColumns [ "Ticks" => frameSeries ]

let completeFrame = baseFrame.Join(data, JoinKind.Left, Lookup.Exact)
completeFrame

Looking at missing values (and making the output a bit prettier) just takes a little helper function.

In [ ]:
let missingCount (frame: Frame<'K, 'C>) =
    frame.Columns
    |> Series.map (fun _ os -> os
                            |> Series.filterAll (fun _ v -> v.IsNone)
                            |> Series.fillMissingWith 0.
                            |> Stats.count
                            |> fun count -> Series.ofObservations [ "Missing Count", float count
                                                                    "Missing %", (float count) / (float os.KeyCount) * 100. ])
    |> Frame.ofColumns
    |> Frame.transpose

missingCount completeFrame

Funny observation! Why is that? We read the values in UTC. Apparently we dropped the "double" values when switching to summer time.

In [ ]:
completeFrame
|> Frame.filterRows (fun _ os -> os.TryGetAs<float>("Value").HasValue |> not)

Doing imputation in time series can be rather tricky. In this case we can go the easy route, though, and opt for simple forward imputation to avoid a lookahead.

In [ ]:
completeFrame?Value <-
    completeFrame?Value
    |> Series.fillMissing Direction.Forward

missingCount completeFrame

The last thing we can do before we take a closer look at the data is to look at some descriptive statistics.

In [ ]:
let describeFrame (frame: Frame<'R, 'C>) =
    Frame.getNumericCols frame
    |> Series.map (fun key s -> Stats.describe s)
    |> Frame.ofRows
    |> Frame.transpose

describeFrame completeFrame

At this point, because we don't have a more advanced time series management system in place I'll stick to a simple CSV file. That's good enough for a Pitch-MVP.

In [ ]:
completeFrame.SaveCsv("../data/at_load_hourly_mw.csv", [ "TimeStamp" ])